Separando ações de interesse

In [7]:
import os
import pandas as pd

input_path = os.path.join('df_final.csv')

Convertendo para os tipos adequados

In [13]:
df = pd.read_csv(input_path, sep=",", parse_dates=["File_Date"], index_col=False)
df = df.drop(["Unnamed: 0"], axis=1)

df["Date"] = df["Date"].astype(str)
df["Time"] = df["Time"].astype(str)
df["Datetime"] = df["Date"] + " " + df["Time"]
df = df.drop(["Date", "Time"], axis=1)

df = df.dropna()
df = df[df["Company"] == "APPLE INC COM"]

df["Datetime"] = pd.to_datetime(df["Datetime"], format="%Y-%m-%d %H:%M:%S")
df["Company"] = df["Company"].astype(str)
df["Transaction_type"] = df["Transaction_type"].astype("category")

print(df.dtypes)

File_Date                datetime64[ns]
Company                          object
Shares_Top_of_Book              float64
Prices_Top_of_Book              float64
Price_of_One_Share              float64
Number_of_Shares_Sold           float64
Transaction_type               category
Datetime                 datetime64[ns]
dtype: object


In [15]:
threshold = 5 #segundos

df['File_Date'] = pd.to_datetime(df['File_Date'])
df['group'] = (df['File_Date'] - df['File_Date'].shift().fillna(df['File_Date'].min())).dt.total_seconds().gt(threshold).cumsum()
df['File_Date'] = df.groupby('group').transform('first')['File_Date']
df.drop('group', axis=1, inplace=True)

df #print

,File_Date,Company,Shares_Top_of_Book,Prices_Top_of_Book,Price_of_One_Share,Number_of_Shares_Sold,Transaction_type,Datetime
2,2024-07-10 12:21:57,APPLE INC COM,336.0,230.88,230.92,7.0,bids,2024-07-10 11:21:40
3,2024-07-10 12:21:57,APPLE INC COM,235.0,230.89,230.92,5.0,bids,2024-07-10 11:21:40
4,2024-07-10 12:21:57,APPLE INC COM,222.0,230.90,230.92,3.0,bids,2024-07-10 11:21:40
9,2024-07-10 12:21:57,APPLE INC COM,130.0,230.91,230.92,1.0,bids,2024-07-10 11:21:40
6,2024-07-10 12:21:57,APPLE INC COM,110.0,230.92,230.91,1.0,bids,2024-07-10 11:21:43
...,...,...,...,...,...,...,...,...
1710764,2024-08-06 17:59:10,APPLE INC COM,200.0,205.57,205.57,100.0,asks,2024-08-06 16:58:15
1710763,2024-08-06 17:59:10,APPLE INC COM,1.0,206.75,205.52,100.0,asks,2024-08-06 16:58:25
1710761,2024-08-06 17:59:10,APPLE INC COM,2.0,207.71,205.51,31.0,asks,2024-08-06 16:58:38
1710762,2024-08-06 17:59:10,APPLE INC COM,8.0,207.23,205.51,100.0,asks,2024-08-06 16:58:38


In [17]:
df = df.sort_values(by=["File_Date", "Datetime"], ascending=[True, False])
print(df.head(10))

            File_Date        Company  Shares_Top_of_Book  Prices_Top_of_Book  \
1 2024-07-10 12:21:57  APPLE INC COM               431.0              230.99   
0 2024-07-10 12:21:57  APPLE INC COM               130.0              230.98   
5 2024-07-10 12:21:57  APPLE INC COM               100.0              230.97   
7 2024-07-10 12:21:57  APPLE INC COM                 6.0              230.95   
8 2024-07-10 12:21:57  APPLE INC COM               100.0              230.96   
6 2024-07-10 12:21:57  APPLE INC COM               110.0              230.92   
2 2024-07-10 12:21:57  APPLE INC COM               336.0              230.88   
3 2024-07-10 12:21:57  APPLE INC COM               235.0              230.89   
4 2024-07-10 12:21:57  APPLE INC COM               222.0              230.90   
9 2024-07-10 12:21:57  APPLE INC COM               130.0              230.91   

   Price_of_One_Share  Number_of_Shares_Sold Transaction_type  \
1              230.94                    1.0          

Gerando relatório com o Ydata Profiling

In [19]:
from ydata_profiling import ProfileReport

output_path = os.path.join('reports', 'profilling.html')


report = ProfileReport(
    df,
    title="Oferta de Ações na Bolsa Estadunidense",
    tsmode=True,
    dataset={
        "description": "Dados de oferta de compra e venda de ações em 2024",
        "copyright_year": 2024
    },
)

report.to_file(output_path)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]